<a href="https://colab.research.google.com/github/loicbi/Databricks/blob/develop/08_Databricks_%7C_Pyspark_Add%2C_Rename_and_Drop_Columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install findspark

In [0]:
!pip install pyspark

In [0]:
import findspark
from pyspark.sql.functions import to_date, col, split, char_length, lit, trim, concat, to_char
# findspark.init()
from pyspark.sql import SparkSession
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()
# spark = SparkSession.builder.appName("app_name").getOrCreate()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
df = spark.read.format('csv').option('header', True)\
          .option('inferSchema', True)\
          .option('sep', ',')\
          .load('/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/Africa Cup of Nations Matches.csv')

df.printSchema()
df.show(10)


## Remove space in columns and rows

In [0]:
df_result = df.withColumn('Year_2', trim(df.Year + 1)).show()

In [0]:
# Create a list of tuples where the first element is the old column name and the second element is the new column name
rename_columns = [(c, c.strip()) for c in df.columns]

# Use the withColumnRenamed method with a dictionary to rename multiple columns

for current_name, new_name in rename_columns:
    df = df.withColumnRenamed(current_name, new_name)
    df = df.withColumn(new_name, trim(df[new_name]))


# Print the schema of the renamed DataFrame
df.printSchema()
df.show(10)

# Add column ===> withColumn()  or withColumns()
## lit() : constant literal


In [0]:
# add new column by calculation
df_add_column = df.withColumn('DateCast_to_YYYY-MM-DD', lit(to_date(df['Date'], 'dd-MMM-yy')))

# add multiple columns
df_add_column = df_add_column.withColumns({'homeTeam # AwayTeam': concat(df_add_column['homeTeam'],  lit(' ') , df_add_column['AwayTeam']), 'ColumnAdded': lit(999999999999999999)})
# + lit(' ') + df_add_column['AwayTeam']
# df_add_column.printSchema()
df_add_column.show(2)

# Rename column ===> withColumnRenamed() or withColumnsRenamed

In [0]:
df_rename_column = df_add_column.withColumnsRenamed({'DateCast_to_YYYY-MM-DD': 'DateCast', 'Attendance': '_renamed__Attendance'})
display(df_rename_column)

# Remove column ===> drop()

In [0]:
df_remove_col = df_rename_column.drop('_renamed__Attendance')
df_remove_col.printSchema()